In [1]:
import pandas as pd

accomodations_clusters = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/clusters_central_location.csv')
venues = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/venues.csv')
time_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/time_matrix.csv')
bus_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/accomodations_to_venues.csv')
bus_terminals = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/bus_terminals.csv')
merged_matrix = pd.read_csv('https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/merged_matrix.csv')

In [2]:
venues = venues.dropna()

In [3]:
import folium

# Create a map centered at the average latitude and longitude
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=10)

# Add markers for each cluster
for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m)


# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m)
    
# Add markers for each bus terminal
for _, row in bus_terminals.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='green')
    ).add_to(m)
# Display the map
m

/Users/leonardogreco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
demand= accomodations_clusters['total_accommodates'].to_numpy()
demand

array([ 6969,  6395,  2801,  7026,  4909,  1545,  6342,  6399,  1171,
        4117,  3110,  3088,  5080,  4779,  2666,  9095,  7555,  4679,
        2111,  3376,  7538,  7068,  6853,  1576,  2974,  2110,  1319,
       10383,  1268,   273,  8063,  8239,  3833,  6408,  7028,  2057,
        4099,  2060,  2646,  2505,  6751,   934,  4119,  9229,   563,
        7296,  2670,  2156,  2263,  1619, 12812])

In [5]:
from ortools.sat.python import cp_model

# Set the number of bus stops to locate
p = 15

# bus_matrix rows: accommodations; columns: candidate bus terminals
accom = list(bus_matrix.index)
bus_stops = list(bus_matrix.columns[1:])

model = cp_model.CpModel()

# Decision variables:
x = {j: model.NewBoolVar(f"x_{j}") for j in bus_stops}
y = {}
for idx, i in enumerate(accom):
    for j in bus_stops:
        y[(idx, j)] = model.NewBoolVar(f"y_{idx}_{j}")

# Constraint: each accommodation must be assigned to exactly one bus stop.
for idx, i in enumerate(accom):
    model.Add(sum(y[(idx, j)] for j in bus_stops) == 1)

# Constraint: assignment only possible if bus stop is selected.
for idx, i in enumerate(accom):
    for j in bus_stops:
        model.Add(y[(idx, j)] <= x[j])

# Constraint: exactly p bus stops are selected.
model.Add(sum(x[j] for j in bus_stops) == p)

# Objective: minimize total (demand-weighted) distance.
# We convert bus_matrix.loc[i, j] to an integer cost.
objective_terms = []
for idx, i in enumerate(accom):
    for j in bus_stops:
        # Multiply demand and the scaled distance.
        cost = int(round(bus_matrix.loc[i, j]))
        objective_terms.append(demand[idx] * cost * y[(idx, j)])
model.Minimize(sum(objective_terms))

# Solve the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    selected = [j for j in bus_stops if solver.Value(x[j]) == 1]
    print("Selected bus stops:", selected)
else:
    print("No solution found.")

Selected bus stops: ['BD14', 'BD15', 'BD16', 'BT16', 'BT03', 'BT08', 'BT24', 'BL14', 'BD04', 'BT11', 'BT06', 'BT21', 'BL20', 'BT18', 'BL23']


In [6]:
#map only selected bus stops ['BD14', 'BD15', 'BT03', 'BL14', 'BT23', 'BT18', 'BL23', 'BL07', 'BT05']
# Add markers for each cluster
map_center = [accomodations_clusters['avg_latitude'].mean(), accomodations_clusters['avg_longitude'].mean()]
m2 = folium.Map(location=map_center, zoom_start=10)

for _, row in accomodations_clusters.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"ID: {row['id']}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
    ).add_to(m2)
# Add markers for each venue
for _, row in venues.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Approx. Capacity: {row['Approx. Capacity']}<br>Venue: {row['Venue']}",
        icon=folium.Icon(color='purple')
    ).add_to(m2)

for _, row in bus_terminals[bus_terminals['id'].isin(selected)].iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"ID: {row['id']}<br>Terminal: {row['FACILITY']}",
        icon=folium.Icon(color='lightgreen', icon='ok-sign')
    ).add_to(m2)
# Display the map with selected bus stops
m2

In [7]:
merged_matrix

,Unnamed: 0,A1,A10,A11,A12,A13,A14,A15,A16,A17,...,BT19,BT20,BT21,BT22,BT23,BT24,BT25,BT26,BT27,BT28
0,A1,0.0,3304.0,3496.0,1727.0,1956.0,2020.0,2410.0,1512.0,3273.0,...,1850.0,2149.0,1860.0,2759.0,2024.0,2357.0,2234.0,1594.0,1485.0,1745.0
1,A10,3263.0,0.0,4901.0,2745.0,2974.0,3033.0,2460.0,2917.0,2778.0,...,1696.0,1627.0,2491.0,2459.0,2890.0,2960.0,2575.0,2406.0,2420.0,2006.0
2,A11,3459.0,4958.0,0.0,3305.0,3534.0,3598.0,4079.0,2432.0,4852.0,...,3477.0,3757.0,3064.0,4313.0,3578.0,3911.0,3788.0,3326.0,3408.0,3477.0
3,A12,1753.0,2784.0,3117.0,0.0,593.0,1014.0,1617.0,1405.0,2390.0,...,1556.0,1785.0,2037.0,1875.0,1140.0,1473.0,1350.0,1783.0,1424.0,1630.0
4,A13,1909.0,2939.0,3272.0,576.0,0.0,489.0,1654.0,1561.0,2426.0,...,1711.0,1940.0,2193.0,1911.0,1176.0,1509.0,1386.0,1938.0,1579.0,1785.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,BT24,2357.0,2960.0,3911.0,1473.0,1509.0,1455.0,1027.0,2040.0,1565.0,...,1847.0,1958.0,2368.0,1327.0,980.0,0.0,934.0,1965.0,1990.0,1634.0
150,BT25,2234.0,2575.0,3788.0,1350.0,1386.0,1332.0,250.0,1918.0,1442.0,...,1126.0,1238.0,1647.0,840.0,857.0,952.0,0.0,1244.0,1479.0,914.0
151,BT26,1594.0,2406.0,3326.0,1783.0,1938.0,2233.0,1496.0,1365.0,2523.0,...,933.0,1044.0,1418.0,1854.0,2018.0,2087.0,1345.0,0.0,357.0,707.0
152,BT27,1485.0,2420.0,3408.0,1424.0,1579.0,1874.0,1556.0,1447.0,2584.0,...,1164.0,1311.0,1731.0,1914.0,1635.0,2047.0,1405.0,362.0,0.0,987.0


In [8]:
merged_matrix['Unnamed: 0'].unique()

array(['A1', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17',
       'A18', 'A19', 'A2', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25',
       'A26', 'A27', 'A28', 'A29', 'A3', 'A30', 'A31', 'A32', 'A33',
       'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A4', 'A40', 'A41',
       'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A5',
       'A50', 'A51', 'A6', 'A7', 'A8', 'A9', 'V1', 'V10', 'V11', 'V12',
       'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V2', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29',
       'V3', 'V30', 'V31', 'V32', 'V33', 'V34', 'V4', 'V5', 'V6', 'V7',
       'V8', 'V9', 'BD01', 'BD02', 'BD03', 'BD04', 'BD05', 'BD06', 'BD07',
       'BD08', 'BD09', 'BD10', 'BD11', 'BD12', 'BD13', 'BD14', 'BD15',
       'BD16', 'BD17', 'BL01', 'BL02', 'BL03', 'BL04', 'BL05', 'BL06',
       'BL07', 'BL08', 'BL09', 'BL10', 'BL11', 'BL12', 'BL13', 'BL14',
       'BL15', 'BL16', 'BL17', 'BL18', 'BL19', 'BL20', 'BL21', 'BL22',
       'BL23'

In [9]:
# Define the lists of location IDs

# Bus stops as determined by your optimization
bus_stop_ids = selected

# Venues: the provided numbers correspond to venue IDs prefixed with 'V'
# For instance, 9 becomes 'V9', 30 becomes 'V30', etc.
venue_ids = ['V9', 'V30', 'V5', 'V31', 'V17', 'V14', 'V24', 'V8', 'V11', 'V12', 'V19', 'V18', 'V28', 'V20']

# Accommodations clusters: use the 'id' column from the dataframe
accom_ids = accomodations_clusters['id'].tolist()

# Combine all location IDs
all_ids = bus_stop_ids + venue_ids + accom_ids

# Set the index of merged_matrix to the column that holds the identifiers
mm_indexed = merged_matrix.set_index('Unnamed: 0')

# Subset the merged matrix to include only rows and columns for the selected locations
new_matrix = mm_indexed.loc[all_ids, all_ids]
new_matrix

,BD14,BD15,BD16,BT16,BT03,BT08,BT24,BL14,BD04,BT11,...,A42,A43,A44,A45,A46,A47,A48,A49,A50,A51
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
BD14,0.0,1947.0,2868.0,1976.0,1858.0,1052.0,1691.0,1614.0,2546.0,1472.0,...,4028.0,1868.0,2199.0,2731.0,1740.0,2078.0,1859.0,1696.0,1981.0,1607.0
BD15,1769.0,0.0,2951.0,1936.0,2129.0,1551.0,2180.0,2062.0,3054.0,1431.0,...,4130.0,1827.0,1880.0,2813.0,1699.0,1621.0,1334.0,1592.0,1788.0,1566.0
BD16,2574.0,2932.0,0.0,2681.0,1420.0,2359.0,2987.0,2910.0,3843.0,2627.0,...,3226.0,2700.0,1826.0,1577.0,2519.0,2383.0,2482.0,2661.0,2490.0,2726.0
BT16,1879.0,2037.0,2695.0,0.0,2130.0,1661.0,1979.0,1309.0,1598.0,1154.0,...,4005.0,215.0,1603.0,2844.0,647.0,1070.0,1250.0,974.0,978.0,871.0
BT03,1684.0,2044.0,1693.0,2049.0,0.0,1469.0,2097.0,2020.0,2953.0,1737.0,...,2850.0,2067.0,1163.0,1624.0,1694.0,1558.0,1657.0,1836.0,1666.0,1873.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A47,2078.0,1621.0,2383.0,1070.0,1558.0,1772.0,2042.0,1360.0,2299.0,1219.0,...,3537.0,1007.0,1228.0,2259.0,687.0,0.0,722.0,795.0,564.0,858.0
A48,1859.0,1334.0,2482.0,1250.0,1657.0,1553.0,2115.0,1432.0,2436.0,1002.0,...,3637.0,1187.0,1328.0,2359.0,883.0,705.0,0.0,458.0,969.0,831.0
A49,1696.0,1592.0,2661.0,974.0,1836.0,1390.0,1669.0,987.0,1991.0,836.0,...,3816.0,874.0,1507.0,2538.0,592.0,719.0,458.0,0.0,889.0,396.0


In [10]:
highest_value = new_matrix.max().max()
highest_value

np.float64(6496.0)

# VRP

In [13]:
# ============================================================
# 0)  CONFIG
# ============================================================
FILE = "new_matrix.csv"          # path to your CSV
MAX_STOPS_PER_ROUTE = 25         # None = unlimited
TIME_LIMIT_SEC      = 120        # OR-Tools search time cap

# ============================================================
# 1)  LOAD & CLEAN THE MATRIX
# ============================================================
import pandas as pd, numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

df  = pd.read_csv(FILE, index_col=0)              # IDs = row index
bad = df.apply(pd.to_numeric, errors="coerce")    # force numeric

if bad.isna().any().any():
    r = bad.index[ bad.isna().any(axis=1) ][0]
    c = bad.columns[ bad.isna().any(axis=0) ][0]
    raise ValueError(f"⚠️ non-numeric cell at row {r!r}, column {c!r}")

distance_matrix = bad.astype(np.int32).to_numpy()   # seconds
node_ids        = df.index.astype(str).tolist()
n_nodes         = len(node_ids)

# ============================================================
# 2)  CLASSIFY NODES
# ============================================================
bus_idx   = [i for i,s in enumerate(node_ids) if s.startswith("B")]
acc_idx   = [i for i,s in enumerate(node_ids) if s.startswith("A")]
venue_idx = [i for i,s in enumerate(node_ids) if s.startswith("V")]

assert bus_idx,   "No IDs starting with 'B' – depots required."
assert acc_idx,   "No IDs starting with 'A' – accommodation required."
assert venue_idx, "No IDs starting with 'V' – venues required."

# ============================================================
# 3)  ROUTING MODEL
# ============================================================
man = pywrapcp.RoutingIndexManager(n_nodes, len(bus_idx), bus_idx, bus_idx)
rt  = pywrapcp.RoutingModel(man)

def seconds_cb(fi, ti):
    f, t = man.IndexToNode(fi), man.IndexToNode(ti)
    return int(distance_matrix[f][t])

transit = rt.RegisterTransitCallback(seconds_cb)
rt.SetArcCostEvaluatorOfAllVehicles(transit)

if MAX_STOPS_PER_ROUTE:
    ones = rt.RegisterUnaryTransitCallback(lambda _: 1)
    rt.AddDimension(ones, 0, MAX_STOPS_PER_ROUTE, True, "Stops")

for v in range(len(bus_idx)):
    rt.SetFixedCostOfVehicle(1_000, v)

# ============================================================
# 4)  SEARCH
# ============================================================
p = pywrapcp.DefaultRoutingSearchParameters()
p.first_solution_strategy    = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
p.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
p.time_limit.seconds         = TIME_LIMIT_SEC

sol = rt.SolveWithParameters(p)

# ============================================================
# 5)  REPORT  (seconds → minutes, include depot return)
# ============================================================
if not sol:
    print("❌  No solution found – relax constraints or raise TIME_LIMIT_SEC.")
else:
    for v in range(len(bus_idx)):
        idx = rt.Start(v)
        if rt.IsEnd(sol.Value(rt.NextVar(idx))):   # unused bus
            continue

        depot_id  = node_ids[man.IndexToNode(idx)]
        hop, time_sec = 0, 0

        print(f"\n🚌  Bus Line {v+1}  (depot {depot_id})")

        while not rt.IsEnd(idx):
            node = man.IndexToNode(idx)
            hop += 1
            print(f"  {hop:2d}. {node_ids[node]}")
            prev = idx
            idx  = sol.Value(rt.NextVar(idx))
            time_sec += distance_matrix[
                man.IndexToNode(prev), man.IndexToNode(idx)
            ]

        # show the depot again to make the return explicit
        hop += 1
        print(f"  {hop:2d}. {depot_id}   (return)")

        if MAX_STOPS_PER_ROUTE:
            stops = sol.Value(rt.GetDimensionOrDie("Stops").CumulVar(idx))
            print(f"     — {stops} stops, ≈ {time_sec/60:.1f} min")
        else:
            print(f"     — ≈ {time_sec/60:.1f} min")


🚌  Bus Line 1  (depot BD14)
   1. BD14
   2. V30
   3. A15
   4. A40
   5. A17
   6. V9
   7. A18
   8. A8
   9. A28
  10. A27
  11. V5
  12. A29
  13. A26
  14. A41
  15. A34
  16. A33
  17. V17
  18. A12
  19. A13
  20. V31
  21. A14
  22. BD14   (return)
     — 21 stops, ≈ 245.8 min

🚌  Bus Line 4  (depot BT16)
   1. BT16
   2. A43
   3. V24
   4. V14
   5. A31
   6. A3
   7. A4
   8. A2
   9. A1
  10. A21
  11. A19
  12. V28
  13. A44
  14. A7
  15. V18
  16. A37
  17. V19
  18. A5
  19. A25
  20. V12
  21. A6
  22. A10
  23. A39
  24. A23
  25. BT16   (return)
     — 24 stops, ≈ 285.7 min

🚌  Bus Line 13  (depot BL20)
   1. BL20
   2. A51
   3. V8
   4. A49
   5. A48
   6. A16
   7. V20
   8. A11
   9. A36
  10. A45
  11. A32
  12. A35
  13. A9
  14. A30
  15. A42
  16. A38
  17. A24
  18. A22
  19. A20
  20. A47
  21. A50
  22. V11
  23. A46
  24. BL20   (return)
     — 23 stops, ≈ 431.7 min


In [19]:
# ============================================================
# 0)  CONFIG
# ============================================================
FILE = "new_matrix.csv"          # distance/time matrix (seconds)
TIME_LIMIT_SEC = 180             # give the solver 3 minutes
MAX_STOPS_PER_ROUTE = 25         # None = unlimited

# exact list & order of depots to use
TERMINALS = [
    "BD14","BD15","BD16","BT16","BT03","BT08","BT24","BL14",
    "BD04","BT11","BT06","BT21","BL20","BT18","BL23"
]

# ============================================================
# 1)  LOAD & CLEAN THE MATRIX
# ============================================================
import pandas as pd, numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

df  = pd.read_csv(FILE, index_col=0)             # IDs in the index
bad = df.apply(pd.to_numeric, errors="coerce")   # force numeric

if bad.isna().any().any():
    r = bad.index[ bad.isna().any(axis=1) ][0]
    c = bad.columns[ bad.isna().any(axis=0) ][0]
    raise ValueError(f"non-numeric cell at row {r!r}, column {c!r}")

dist_mat = bad.astype(np.int32).to_numpy()       # seconds
ids      = df.index.astype(str).tolist()
n_nodes  = len(ids)

# ------------------------------------------------------------
# make sure every requested depot exists in the CSV
missing = [d for d in TERMINALS if d not in ids]
assert not missing, f"Depot(s) not found in CSV: {missing}"

# OR-Tools wants the depot indices
bus_idx = [ids.index(d) for d in TERMINALS]

# classify the other nodes (not strictly needed, but sanity)
acc_idx   = [i for i,s in enumerate(ids) if s.startswith("A")]
venue_idx = [i for i,s in enumerate(ids) if s.startswith("V")]
assert acc_idx and venue_idx, "Need at least one A- and one V- node"

# ============================================================
# 2)  ROUTING MODEL
# ============================================================
man = pywrapcp.RoutingIndexManager(n_nodes, len(bus_idx), bus_idx, bus_idx)
rt  = pywrapcp.RoutingModel(man)

# transit (cost) = travel seconds
def sec_cb(fi, ti):
    f, t = man.IndexToNode(fi), man.IndexToNode(ti)
    return int(dist_mat[f][t])

transit = rt.RegisterTransitCallback(sec_cb)
rt.SetArcCostEvaluatorOfAllVehicles(transit)

# limit total stops if desired
if MAX_STOPS_PER_ROUTE:
    ones = rt.RegisterUnaryTransitCallback(lambda _: 1)
    rt.AddDimension(ones, 0, MAX_STOPS_PER_ROUTE, True, "Stops")

# ---- force every bus to be used (≥ 1 real stop) ---------------------------
if MAX_STOPS_PER_ROUTE:
    stop_dim = rt.GetDimensionOrDie("Stops")
else:
    # create a tiny 1-per-hop dimension just for this purpose
    ones = rt.RegisterUnaryTransitCallback(lambda _: 1)
    rt.AddDimension(ones, 0, 10**6, True, "MustUse")
    stop_dim = rt.GetDimensionOrDie("MustUse")

solver = rt.solver()
for v in range(len(bus_idx)):
    end_cumul = stop_dim.CumulVar(rt.End(v))
    solver.Add(end_cumul >= 1)      # at least one hop besides depot
# (leave vehicle fixed cost at 0 – no penalty, no reward)

# ============================================================
# 3)  SEARCH PARAMETERS
# ============================================================
p = pywrapcp.DefaultRoutingSearchParameters()
p.first_solution_strategy    = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
p.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
p.time_limit.seconds         = TIME_LIMIT_SEC

solution = rt.SolveWithParameters(p)

# ============================================================
# 4)  SINGLE-LINE REPORT  (seconds → minutes)
# ============================================================
if not solution:
    print("❌  No solution found — try increasing TIME_LIMIT_SEC "
          "or relaxing MAX_STOPS_PER_ROUTE.")
else:
    for v, depot_node in enumerate(bus_idx):
        idx = rt.Start(v)
        route = [ids[depot_node]]          # start depot
        time_sec = 0

        while not rt.IsEnd(idx):
            prev = idx
            idx  = solution.Value(rt.NextVar(idx))
            route.append(ids[man.IndexToNode(idx)])
            time_sec += dist_mat[man.IndexToNode(prev), man.IndexToNode(idx)]

        stops = len(route) - 2              # minus start & return depot
        minutes = time_sec / 60
        print(f"{ids[depot_node]:<5}: " + " → ".join(route)
              + f"   ({stops} stops, {minutes:.1f} min)")

BD14 : BD14 → A14 → V31 → A13 → A12 → V17 → A33 → A34 → A2 → A4 → A3 → A41 → A27 → V5 → A29 → A26 → A40 → A15 → A18 → V30 → BD14   (19 stops, 219.2 min)
BD15 : BD15 → BD15   (0 stops, 0.0 min)
BD16 : BD16 → BD16   (0 stops, 0.0 min)
BT16 : BT16 → A43 → V24 → V14 → A51 → V8 → A49 → A48 → A47 → A46 → V11 → A50 → A7 → V18 → A37 → V19 → A5 → A25 → V12 → A6 → A10 → A39 → A23 → BT16   (22 stops, 249.3 min)
BT03 : BT03 → BT03   (0 stops, 0.0 min)
BT08 : BT08 → BT08   (0 stops, 0.0 min)
BT24 : BT24 → A8 → BT24   (1 stops, 17.0 min)
BL14 : BL14 → BL14   (0 stops, 0.0 min)
BD04 : BD04 → A17 → V9 → BD04   (2 stops, 26.6 min)
BT11 : BT11 → A31 → BT11   (1 stops, 6.4 min)
BT06 : BT06 → A28 → BT06   (1 stops, 8.6 min)
BT21 : BT21 → BT21   (0 stops, 0.0 min)
BL20 : BL20 → BL20   (0 stops, 0.0 min)
BT18 : BT18 → A21 → A1 → A19 → V28 → A44 → A9 → A30 → A42 → A38 → A24 → A35 → A32 → A11 → A36 → A45 → V20 → A16 → A22 → A20 → BT18   (19 stops, 422.7 min)
BL23 : BL23 → BL23   (0 stops, 0.0 min)
